In [ ]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [ ]:
accidents = pd.read_csv('accidents.csv')
prices = pd.read_csv('prices.csv')

In [ ]:
def drop_columns(df, columns):
  df = df.drop(columns, axis=1)
  return df

In [ ]:
columns_to_drop = [
        'Report Number', 'Supplemental Number', 'Accident Year', 'Operator ID', 'Operator Name',
       'Pipeline/Facility Name', 'Pipeline Location', 'Pipeline Type',
       'Liquid Type', 'Liquid Subtype', 'Liquid Name', 'Accident City',
       'Accident County', 'Accident State', 'Accident Latitude',
       'Accident Longitude', 'Cause Category', 'Cause Subcategory',
       'Unintentional Release (Barrels)', 'Intentional Release (Barrels)',
       'Liquid Recovery (Barrels)', 'Liquid Ignition',
       'Liquid Explosion', 'Pipeline Shutdown', 'Shutdown Date/Time',
       'Restart Date/Time', 'Public Evacuations', 'Operator Employee Injuries',
       'Operator Contractor Injuries', 'Emergency Responder Injuries',
       'Other Injuries', 'Public Injuries', 'All Injuries',
       'Operator Employee Fatalities', 'Operator Contractor Fatalities',
       'Emergency Responder Fatalities', 'Other Fatalities',
       'Public Fatalities', 'Property Damage Costs',
       'Lost Commodity Costs', 'Public/Private Property Damage Costs',
       'Emergency Response Costs', 'Environmental Remediation Costs',
       'Other Costs']
accidents = drop_columns(accidents, columns_to_drop)

In [ ]:
def find_price(date, prices):
  for j in range(len(prices)):
      if (str(date) == str(prices['date'][j])):
        return prices['price'][j]
  return False

In [ ]:
def make_prices(prices, accidents):
  combined = pd.DataFrame(columns=["date", "barrels", "fatalities", "cost", "price", "next_day_price", "next_week_price", "next_day", "next_week"])
  for i in range(len(accidents)):
    date = datetime.strptime(str(accidents['Accident Date/Time'][i]), '%m/%d/%Y %H:%M %p').date()
    date2 = date
    date3 = date + timedelta(days=1)
    date4 = date + timedelta(days=7)
    while True:
      price = find_price(date2, prices)
      next_day_price = find_price(date3, prices)
      next_week_price = find_price(date4, prices)
      if date3 > datetime.now().date(): next_day_price = 1
      if date4 > datetime.now().date(): next_week_price = 1
      if price and next_day_price and next_week_price: break
      if price == False: date2 -= timedelta(days=1)
      if next_day_price == False: date3 += timedelta(days=1)
      if next_week_price == False: date4 += timedelta(days=1)
    if float(price) > float(next_day_price): next_day = 0
    else: next_day = 1
    if float(price) > float(next_week_price): next_week = 0
    else: next_week = 1
    combined = combined.append({
        "date": date,
        "barrels": accidents['Net Loss (Barrels)'][i],
        "fatalities": accidents['All Fatalities'][i],
        "cost": accidents['All Costs'][i],
        "price": price,
        "next_day_price": next_day_price,
        "next_week_price": next_week_price,
        "next_day": next_day,
        "next_week": next_week
        }, ignore_index=True)
  return combined

In [ ]:
accidents = make_prices(prices, accidents)

In [ ]:
accidents.to_csv('combined.csv', index=False)